In [1]:
from transformers import VivitForVideoClassification, VivitImageProcessor
import torch

# Video pre loader
model_video = VivitForVideoClassification.from_pretrained("google/vivit-b-16x2-kinetics400")
processor_video = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")

In [2]:
video = torch.randint(low=0, high=256, size=(32, 3, 224, 224))
video = processor_video(list(video), return_tensors="pt")

In [3]:
video.keys()

dict_keys(['pixel_values'])

In [4]:
patched_video = model_video(**video).logits.squeeze(0)

In [5]:
patched_video.shape

torch.Size([400])

In [6]:
from constrastive_model import ContrastiveModel

model = ContrastiveModel(hidden_channels=200, out_channels=100)
model.load_state_dict(torch.load("../../dependencies/VATE/best_model_contrastive.pt"))
model.eval()

ContrastiveModel(
  (embedding_video): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=100, bias=True)
  )
  (embedding_audio): Sequential(
    (0): Linear(in_features=768, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=100, bias=True)
  )
  (embedding_text): Sequential(
    (0): Linear(in_features=768, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=100, bias=True)
  )
)

In [7]:
res = model(x_video=patched_video.unsqueeze(0), x_audio=None, x_text=None)

In [9]:
res[0].shape

torch.Size([1, 100])